# Momentum resolution

In the following, all measurements are in metres, resp. radians.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

## Tracking resolution

### Single trajectory

In [ ]:
n = 5
delta_z = 0.02

z0 = 0

np.random.seed(14)
x0_true = np.random.normal(0, 0.001)
s0_true = np.random.normal(0, 0.1)

x0 = x0_true

def find_x_position(z_diff, s0):
    return np.tan(s0) * z_diff

def find_cell(x_pos):
    return np.floor(x_pos / 0.0005)

cells_hit = np.full(n, np.nan)

for i in range(1, n+1):
    cells_hit[i] = find_cell(find_x_position(delta_z * i))



